COMP 215 - LAB 2 (NEO)
----------------
#### Name: Shayan Heidari
#### Date: 2023-01-16

This lab exercise is mostly a review of strings, tuples, lists, dictionaries, and functions.

**Building on new concepts from lab 1**:
  * `datetime.date` objects represent a calendar date
  * *list comprehension* provides a compact way to represent map and filter algorithms

**New Python Concepts**:
  * *f-string* simplifies string formatting operations

As usual, the first code cell simply imports all the modules we'll be using...

In [1]:
import datetime, json, requests
from pprint import pprint    # Pretty Print - built-in python function to nicely format data structures
from dataclasses import dataclass

We'll answer some questions about [Near Earth Objects](https://cneos.jpl.nasa.gov/)
> using NASA's API:  [https://api.nasa.gov/](https://api.nasa.gov/#NeoWS)

You should register for your own API key, (but may use the DEMO_KEY to get started).

First we need a short tutorial on python dates and [f-strings](https://realpython.com/python-f-strings/)...

In [2]:
today = datetime.date.today()   # get a date object representing today's date
print(today, type(today))
formatted_date = f'Today is: {today}'   # A format string - notice how the variable `today` is formatted into the stringg
print(formatted_date)

2023-02-04 <class 'datetime.date'>
Today is: 2023-02-04


### Make a query

Let's get some data from the NEO database...
Here's a query that gets the observation "feed" for today.
(Note: I hard-coded the date below to lock down the data for the lab - ideally used `today()` so the notebook is always up-to-date.)

# change to demo API for submission


In [3]:
API_KEY = 'HXKCZVSZS0dUr6rcfPO4N76u0LOueCavzMxReGwp'  # substitute your API key here

today = str(datetime.date.today() - datetime.timedelta(days = 1))  #  Using yesterday since data for current day might not be available yet
# Use an f-string here to "format" the date and API key varaibles.
url = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={today}&end_date={today}&api_key={API_KEY}'

response = requests.request("GET", url, headers={}, data={})

data = json.loads(response.text)  # recall json.loads for lab 1

# TIP: print(data) to see the whole data structure returned, here we grab just the list of NEO's:
n_results = data['element_count']
neos = data['near_earth_objects'][today]
print(f'{n_results} Near Earth Objects found for {today}')
pprint(neos[:3])
#print(json.dumps(response.text))

16 Near Earth Objects found for 2023-02-03
[{'absolute_magnitude_h': 18.33,
  'close_approach_data': [{'close_approach_date': '2023-02-03',
                           'close_approach_date_full': '2023-Feb-03 22:49',
                           'epoch_date_close_approach': 1675464540000,
                           'miss_distance': {'astronomical': '0.1159826622',
                                             'kilometers': '17350759.222049514',
                                             'lunar': '45.1172555958',
                                             'miles': '10781261.8468267332'},
                           'orbiting_body': 'Earth',
                           'relative_velocity': {'kilometers_per_hour': '46176.7934314699',
                                                 'kilometers_per_second': '12.8268870643',
                                                 'miles_per_hour': '28692.4636741264'}}],
  'estimated_diameter': {'feet': {'estimated_diameter_max': 4207.5087712198,
   

Next we extract just the potentially hazerdous asteroids, using a Comp115-style list accumulator *loop*:

In [4]:
hazards =  []
for item in neos:
  if item['is_potentially_hazardous_asteroid'] is True:
    hazards.append(item)
print(f'{len(hazards)} potentially hazardous asteroids identified.')

1 potentially hazardous asteroids identified.


## Exercise 1

In the code cell below, **re-write the accumulator loop above** as a [list comprehension](https://realpython.com/lessons/list-comprehensions-overview/) that implements a ["filter"](https://youtu.be/hUes6y2b--0)
Notice how this provides a concise way to "filter" items of interest from a larger data set.

In [5]:
# Your code here
hazards = [item for item in neos if item['is_potentially_hazardous_asteroid'] is True]
pprint(hazards[:5])

[{'absolute_magnitude_h': 21.88,
  'close_approach_data': [{'close_approach_date': '2023-02-03',
                           'close_approach_date_full': '2023-Feb-03 08:51',
                           'epoch_date_close_approach': 1675414260000,
                           'miss_distance': {'astronomical': '0.0121452393',
                                             'kilometers': '1816901.929920291',
                                             'lunar': '4.7244980877',
                                             'miles': '1128970.5081943758'},
                           'orbiting_body': 'Earth',
                           'relative_velocity': {'kilometers_per_hour': '35705.6770230417',
                                                 'kilometers_per_second': '9.9182436175',
                                                 'miles_per_hour': '22186.1191480117'}}],
  'estimated_diameter': {'feet': {'estimated_diameter_max': 820.3988255998,
                                  'estimated_diamet

## Fetch Complete Data for One Asteroid

Notice that the record for each `neo` is a dictionary with `id` field that uniquely identifies this record in the database.

We can use this `id` to fetch complete orbital and close approach data for the NEO.

For example, this query fetches the complete data set for the first hazardous asteroid...


In [ ]:
id = hazards[0]['id']
url = f'https://api.nasa.gov/neo/rest/v1/neo/{id}/?api_key={API_KEY}'
response = requests.request("GET", url, headers={}, data={})
data = json.loads(response.text)

pprint(data)

Notice that the `miss_distance` field contains the distance (in various units) by which the NEO missed an "orbiting body".

## Exercise 2

In the code cell below, write a python function that takes a list of "close approach data" as a parameter,
and returns a 2-tuple with the (date, miss km) of the closest approach to Earth in the list (where "miss km" is the miss distance in km).

Hints:
* notice the input is a list of dictionaries.  Each dictionary has a 'close_approach_date", "orbiting_body", and 'miss_distance' field.
* we are only interested in the closest approach to "Earth"
* use a loop if that is easier to understand - we will look at more compact algorithms to solve this problem in class.

Add at least one unit test to check your work - note the test data only needs dictionaries with the fields your function actually uses.


In [7]:
# your code here

nasaApproachData = data['close_approach_data']

def closestDeath(data):
    earthOrbitter = [orbitter for orbitter in data if orbitter['orbiting_body'] == "Earth"]

    closest = float(earthOrbitter[0]['miss_distance']['kilometers']) # set the initial value to something in the list
    
    for i in earthOrbitter:
        distance = float(i['miss_distance']['kilometers'])
        if closest >= distance:
            closest = distance
            nearEarthObject = (i['close_approach_date'], closest)
    return nearEarthObject


assert closestDeath(nasaApproachData[5:10]) == ('1946-02-08', 4702983.950672007) # taking a manual look at the first few, this is the closest and it passes assertion

print(closestDeath(nasaApproachData[5:10]))


('1946-02-08', 4702983.950672007)


## OPTIONAL - Take your skills to the next level...
## Exercise 3

In the code cell below, write a complete program that:
 1. fetches the list of NEO's for this week.
 2. for each NEO, fetch it's complete orbital data and determine its closest approach to Earth
 3. identify which NEO from this week's data makes the closet approach to earth
 4. print a nice message with information about the NEO, which it will approach the Earth, and how close it will come.

Hints:
* you'll need the start and end date - end date is today, see if you can use a [`timedelta`](https://docs.python.org/3/library/datetime.html#timedelta-objects) object to ge the start date (you can do basic "date math" with `timedelta` and `date` objects!)
* you may need to modify the function we wrote in Ex. 2 to return a triple with the NEO's id included;
* lots of opportunity here for more practice with list comprehensions


In [8]:
# your code here


# Lab 3

In [9]:
@dataclass
class Asteroid:
    neoID: str
    name: str
    estimated_diameter: float
    approachData: list

    
    @classmethod
    def from_NEO(cls, neo_id):
        url = f'https://api.nasa.gov/neo/rest/v1/neo/{neo_id}/?api_key={API_KEY}'
        response = requests.request("GET", url, headers={}, data={})
        data = json.loads(response.text)
        records = data['close_approach_data']
        neoID = neo_id
        name = data['name']
        estimated_diameter = float(data['estimated_diameter']['meters']['estimated_diameter_max'])
        
        
        approachData = [CloseApproach.closeData(record, Asteroid(neoID,
        name,
        estimated_diameter,[])) for record in records] # Looks ugly but it has to be an empty list
        
        return Asteroid(neoID, name, estimated_diameter, approachData)
    
    def __str__(self):
        return f'An asteroid object with the ID {self.neoID}, name {self.name}, estimated diameter {self.estimated_diameter}\n'

@dataclass
class CloseApproach:
    asteroid: Asteroid
    orbiting_body: str
    approach_date: datetime.datetime
    miss_distance: float
    relative_velocity: float
    
    @classmethod
    def closeData(cls, neoRecord, Asteroid):
        orbiting_body = neoRecord['orbiting_body']
        approach_date = datetime.datetime.fromisoformat(neoRecord['close_approach_date']).date() # it's always in the iso format
        miss_distance = float(neoRecord['miss_distance']['kilometers'])
        relative_velocity = float(neoRecord['relative_velocity']['kilometers_per_second'])

        return CloseApproach(Asteroid, orbiting_body, approach_date, miss_distance, relative_velocity)

    
astrid = Asteroid.from_NEO('3449054')
print(astrid)
pprint(astrid)

An asteroid object with the ID 3449054, name (2009 DL46), estimated diameter 237.7059095613

Asteroid(neoID='3449054',
         name='(2009 DL46)',
         estimated_diameter=237.7059095613,
         approachData=[CloseApproach(asteroid=Asteroid(neoID='3449054',
                                                       name='(2009 DL46)',
                                                       estimated_diameter=237.7059095613,
                                                       approachData=[]),
                                     orbiting_body='Earth',
                                     approach_date=datetime.date(1904, 5, 26),
                                     miss_distance=3789280.163182445,
                                     relative_velocity=6.6773826693),
                       CloseApproach(asteroid=Asteroid(neoID='3449054',
                                                       name='(2009 DL46)',
                                                       estimated_diamete